## Setup

In [1]:
GLOBAL_SEED = 42

import os
os.environ['PYTHONHASHSEED'] = str(GLOBAL_SEED)

import numpy as np # linear algebra
from numpy import random as np_rnd
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import random as rnd
import pickle
import gc
import time
from itertools import product

from sklearn import linear_model as lm
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics as skl_metrics

from PIL import Image
from torch.utils.data import TensorDataset, Dataset
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW
from transformers import get_polynomial_decay_schedule_with_warmup
from transformers import AutoFeatureExtractor, AutoImageProcessor, AutoModel
from torchvision import transforms
from torchvision.io import read_image

import matplotlib.pyplot as plt
import seaborn as sns

if torch.cuda.is_available():
    print("GPU is ready")
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

import warnings
warnings.filterwarnings(action='ignore')

c:\Users\USER\AppData\Local\Programs\Python\Python37\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


GPU is ready


In [2]:
def seed_everything(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    # python random
    rnd.seed(seed)
    # numpy random
    np_rnd.seed(seed)
    # RAPIDS random
    try:
        cupy.random.seed(seed)
    except:
        pass
    # tf random
    try:
        tf_rnd.set_seed(seed)
    except:
        pass
    # pytorch random
    try:
        torch.backends.cudnn.benchmark = False
        torch.backends.cudnn.deterministic = True
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    except:
        pass

def pickleIO(obj, src, op="r"):
    if op=="w":
        with open(src, op + "b") as f:
            pickle.dump(obj, f)
    elif op=="r":
        with open(src, op + "b") as f:
            tmp = pickle.load(f)
        return tmp
    else:
        print("unknown operation")
        return obj
    
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print('Error: Creating directory. ' + directory)

def findIdx(data_x, col_names):
    return [int(i) for i, j in enumerate(data_x) if j in col_names]

def diff(first, second):
    second = set(second)
    return [item for item in first if item not in second]

In [3]:
class CFG:
    debug = False
    img_category = "person"
    
    n_folds = 5
    img_size = 384
    channels = 3
    epochs = 2 if debug else 30
    early_stopping_rounds = 10
    batch_size = 4
    eta = 5e-4
    weight_decay = 1e-4
    aug = False

In [4]:
label_mapper = {
    "house": {
        'door_yn':{'n':0, 'y':1}, # 문 유무
        'loc':{'left':0, 'center':1, 'right':2}, # 위치
        'roof_yn':{'y':1, 'n':0}, # 지붕 유무
        'window_cnt':{'absence':0, '1 or 2':1, 'more than 3':2}, # 창문 갯수
        'size':{'small':0, 'middle':1, 'big':2}, # 크기
    },
    "tree": {
        "branch_yn": {"n": 0, "y": 1}, # 가지 유무
        "root_yn": {"n": 0, "y": 1}, # 뿌리 유무
        "crown_yn": {"n": 0, "y": 1}, # 수관 유무
        "fruit_yn": {"n": 0, "y": 1}, # 열매 유무
        "gnarl_yn": {"n": 0, "y": 1}, # 옹이나상처 유무
        "loc": {"left": 0, "center": 1, "right": 2}, # 위치
        "size": {"small": 0, "middle": 1, "big": 2}, # 크기
    },
    "person": {
        "eye_yn": {"n": 0, "y": 1}, # 눈 유무
        "leg_yn": {"n": 0, "y": 1}, # 다리 유무
        "loc": {"left": 0, "center": 1, "right": 2}, # 위치
        "mouth_yn": {"n": 0, "y": 1}, # 입 유무
        "size": {"small": 0, "middle": 1, "big": 2}, # 크기
        "arm_yn": {"n": 0, "y": 1}, # 팔 유무
    }
}

## Loading data

In [5]:
df_full = pickleIO(None, "./dataset/prep/df_metainfo.pkl", "r")[CFG.img_category]
df_full["img_path"] = f"./dataset/raw/full_images/" + df_full["id"].astype("str") + ".jpg"

In [6]:
df_full.head()

,id,eye_yn,leg_yn,loc,mouth_yn,size,arm_yn,fold_eye_yn,fold_leg_yn,fold_loc,fold_mouth_yn,fold_size,fold_arm_yn,img_path
0,24_175_23002_person,1,0,0,1,1,0,4,2,2,3,1,2,./dataset/raw/full_images/24_175_23002_person.jpg
1,24_175_23003_person,1,0,1,1,1,1,4,2,4,2,2,0,./dataset/raw/full_images/24_175_23003_person.jpg
2,24_175_23004_person,1,1,2,0,1,1,0,1,0,4,2,2,./dataset/raw/full_images/24_175_23004_person.jpg
3,24_175_23005_person,1,1,1,1,1,1,3,3,2,3,0,4,./dataset/raw/full_images/24_175_23005_person.jpg
4,24_175_23006_person,1,1,2,1,1,1,0,3,1,1,0,4,./dataset/raw/full_images/24_175_23006_person.jpg


## Define model & Training

In [7]:
def get_optimizer_params(model, eta, weight_decay):
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_parameters = [
        # apply weight decay
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
         'lr': eta, 'weight_decay': weight_decay},
        # don't apply weight decay for LayerNormalization layer
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
         'lr': eta, 'weight_decay': 0.0},
    ]
    return optimizer_parameters


def get_scheduler(optimizer, num_warmup_steps, num_training_steps):
    scheduler = get_polynomial_decay_schedule_with_warmup(
        optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps, power=0.5, lr_end=1e-7
    )
    return scheduler


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

In [8]:
def train_fn(model, dl, criterion, optimizer, scheduler, grad_scaler):
    model.train()
    metrics = {
        "loss": AverageMeter("loss", fmt=":.5f"),
        "accuracy": AverageMeter("accuracy", fmt=":.5f"),
        "f1": AverageMeter("accuracy", fmt=":.5f"),
    }
    
    for idx, batch in enumerate(dl):
        if CFG.aug:
            feature = nn.Sequential(
                transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
                transforms.RandomRotation((0, 5)),
            )(batch["feature"])
        else:
            feature = batch["feature"]
        with torch.no_grad():
            feature = model_backbone(feature_extractor(feature, return_tensors="pt")["pixel_values"].to(device)).pooler_output
        with torch.cuda.amp.autocast():
            output = model(feature)
            output = torch.squeeze(output, dim=-1)
            label = batch["label"].to(device)
            loss = criterion(output, label)
        # initialization gradients to zero
        optimizer.zero_grad()
        # get scaled gradients by float16 (default)
        grad_scaler.scale(loss).backward()
        # apply original gradients (unscaling) to parameters
        # if these gradients do not contain infs or NaNs, optimizer.step() is then called.
        # otherwise, optimizer.step() is skipped.
        grad_scaler.step(optimizer)
        grad_scaler.update()
        # calcuate metrics
        metrics["loss"].update(loss.item())

        # if CFG.debug:
        #     if idx >= 10:
        #         break
    
    # update scheduler
    scheduler.step()
    
    del idx, batch, output, label
    gc.collect()
    torch.cuda.empty_cache()
    return metrics

def valid_fn(model, dl, criterion):
    model.eval()
    metrics = {
        "loss": AverageMeter("loss", fmt=":.5f"),
        "accuracy": AverageMeter("accuracy", fmt=":.5f"),
        "f1": AverageMeter("accuracy", fmt=":.5f"),
    }
    
    for idx, batch in enumerate(dl):
        with torch.no_grad():
            feature = model_backbone(feature_extractor(batch["feature"], return_tensors="pt")["pixel_values"].to(device)).pooler_output
        with torch.no_grad():
            output = model(feature.to(device))
            output = torch.squeeze(output, dim=-1)
            label = batch["label"].to(device)
            loss = criterion(output, label)
        # calcuate metrics
        metrics["loss"].update(loss.item())
        
        # if CFG.debug:
        #     if idx >= 10:
        #         break
    
    del idx, batch, output, label
    gc.collect()
    torch.cuda.empty_cache()
    return metrics

In [9]:
def do_training(model, train_dl, valid_dl, criterion):
    # set loss & optimizer
    optimizer_parameters = get_optimizer_params(
        model,
        eta=CFG.eta,
        weight_decay=CFG.weight_decay
    )
    optimizer = AdamW(optimizer_parameters, lr=CFG.eta, weight_decay=CFG.weight_decay)
    scheduler = get_scheduler(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=CFG.epochs
    )
    grad_scaler = torch.cuda.amp.GradScaler()
    
    best_score = np.inf
    early_stopping_cnt = 0
    for epoch in range(CFG.epochs):

        epoch_start_time = time.time()
        
        # training
        train_metrics = train_fn(model, train_dl, criterion, optimizer, scheduler, grad_scaler)
        # evaluation
        valid_metrics = valid_fn(model, valid_dl, criterion)
        score = valid_metrics["loss"].avg

        print("Epoch[{0}/{1}]\n train loss : {2}\n train accuracy : {3}\n train f1 : {4}\n valid loss : {5}\n valid accuracy : {6}\n valid f1 : {7}\n eta : {8}\n Elapsed : {9}\n"
              .format(
                  epoch+1, CFG.epochs,
                  round(train_metrics["loss"].avg, 5), round(train_metrics["accuracy"].avg, 5), round(train_metrics["f1"].avg, 5),
                  round(valid_metrics["loss"].avg, 5), round(valid_metrics["accuracy"].avg, 5), round(valid_metrics["f1"].avg, 5),
                  round(optimizer.param_groups[-1]['lr'], 5), round(time.time() - epoch_start_time, 3)
              )
        )
    
        if score < best_score:
            best_score = score
            return_score_dic = {
                "fold": fold,
                "train_loss": train_metrics["loss"].avg,
                "valid_loss": valid_metrics["loss"].avg,
                "train_accuracy": train_metrics["accuracy"].avg,
                "valid_accuracy": valid_metrics["accuracy"].avg,
                "train_f1": train_metrics["f1"].avg,   
                "valid_f1": valid_metrics["f1"].avg,              
            }
            print("INFO: Found best weight\n\n")
            torch.save(
                {'model': model.state_dict()},
                f"{architecture_root_path}weights/{target}_model_fold{fold}_best.pth",
            )
            early_stopping_cnt = 0
        else:
            early_stopping_cnt += 1
        
        if early_stopping_cnt == CFG.early_stopping_rounds:
            break
    
    model.load_state_dict(torch.load(f"{architecture_root_path}weights/{target}_model_fold{fold}_best.pth")["model"])
    
    return return_score_dic

In [10]:
class CustomDataset(Dataset):
    def __init__(self, feature, label, loss_type="binary"):
        self.feature = torch.from_numpy(feature).to(torch.float32)
        self.label = torch.from_numpy(label).to(torch.float32 if loss_type == "binary" else torch.int64)
        
    def __len__(self):
        return len(self.feature)

    def __getitem__(self, idx):
        return {"feature": self.feature[idx], "label": self.label[idx]}

class DNN_CustomModel(nn.Module):
    def __init__(self, params):
        super().__init__()
        self.lin = nn.Sequential(
            nn.Dropout(),
            nn.Linear(params["n_input"], params["hidden_layers"]),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(params["hidden_layers"], params["hidden_layers"] // 4),
            nn.ReLU(),
        )
        self.classifier = nn.Linear(params["hidden_layers"] // 4, params["n_classes"])

    def forward(self, feature):
        feature = self.lin(feature)
        feature = self.classifier(feature)
        return feature

class DNN_TreeClassifier():
    def __init__(self, model_params, model_threshold_params, model_backbone, feature_extractor):
        self.model_list = []
        self.model_threshold_spliter = []
        self.eval_score_list = []
        self.threshold_eval_score_list = []
        self.n_classes = model_params["n_classes"]
        self.model_backbone = model_backbone
        self.feature_extractor = feature_extractor
        self.criterion = nn.BCEWithLogitsLoss() if model_params["n_classes"] == 1 else nn.CrossEntropyLoss()
        for idx, params in enumerate(product(*model_threshold_params["dynamic_params"].values())):
            tmp_params = model_threshold_params["fixed_params"].copy()
            tmp_params["random_state"] = idx
            tmp_params.update({k: v for k, v in zip(model_threshold_params["dynamic_params"].keys(), params)})
            self.model_threshold_spliter.append(DecisionTreeClassifier(**tmp_params))
        self.model_list.append(DNN_CustomModel(model_params))
    def fit(self, x, y, eval_x, eval_y):
        train_dl = DataLoader(CustomDataset(x, y, loss_type="binary" if self.n_classes == 1 else "multiclass"), shuffle=True, batch_size=CFG.batch_size, drop_last=True)
        print(f"INFO: number of iteration : {len(train_dl)}\n")
        valid_dl = DataLoader(CustomDataset(eval_x, eval_y, loss_type="binary" if self.n_classes == 1 else "multiclass"), shuffle=False, batch_size=CFG.batch_size)
        for model in self.model_list:
            model.to(device)
            model.train()
            do_training(model, train_dl, valid_dl, self.criterion)
            model.eval()
            y_prob = self.do_infer(model, eval_x)
            self.eval_score_list.append(skl_metrics.log_loss(eval_y, y_prob))
        x_pred = self.do_infer(self.model_list[np.argmin(self.eval_score_list)], x)
        eval_x_pred = self.do_infer(self.model_list[np.argmin(self.eval_score_list)], eval_x)
        for model in self.model_threshold_spliter:
            model.fit(x_pred, y)
            y_prob = model.predict_proba(eval_x_pred)
            self.threshold_eval_score_list.append(skl_metrics.f1_score(eval_y, y_prob.argmax(axis=1), average="macro"))
    def predict_proba(self, x):
        output = {"y_prob": self.do_infer(self.model_list[np.argmin(self.eval_score_list)], x)}
        output["y_prob_threshold_opt"] = self.model_threshold_spliter[np.argmax(self.threshold_eval_score_list)].predict_proba(output["y_prob"])
        return output
    def do_infer(self, model, x, force_to_cpu=False):
        model.eval()
        model.to(torch.device("cpu") if force_to_cpu else device)
        test_dl = DataLoader(CustomDataset(x, np.zeros(len(x)), loss_type="binary" if self.n_classes == 1 else "multiclass"), shuffle=False, batch_size=CFG.batch_size)
        y_pred = []
        for batch in test_dl:
            with torch.no_grad():
                feature = self.model_backbone(self.feature_extractor(batch["feature"], return_tensors="pt")["pixel_values"].to(torch.device("cpu") if force_to_cpu else device)).pooler_output
            with torch.no_grad():
                output = model(feature)
            if output.shape[-1] == 1:
                output = output.sigmoid()
                output = torch.cat([1 - output, output], dim=-1)      
            else:
                output = F.softmax(output, dim=-1)
            y_pred.append(output.detach().cpu().numpy())
        model.to(torch.device("cpu"))
        return np.concatenate(y_pred)

In [11]:
architecture_name = "swinV1_dnn_relu_noAug_v1"
architecture_root_path = f"./architecture/{architecture_name}/{CFG.img_category}/"
createFolder(architecture_root_path)
createFolder(f"{architecture_root_path}weights/")

In [12]:
feature_extractor = AutoImageProcessor.from_pretrained("microsoft/swin-base-patch4-window12-384-in22k")
model_backbone = AutoModel.from_pretrained("microsoft/swin-base-patch4-window12-384-in22k")
for name, param in model_backbone.named_parameters():
    param.requires_grad = False
model_backbone.eval()
model_backbone.to(device)
print("backbone model is loaded !")

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
Some weights of the model checkpoint at microsoft/swin-base-patch4-window12-384-in22k were not used when initializing SwinModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing SwinModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing SwinModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


backbone model is loaded !


In [13]:
resizer = transforms.Resize((CFG.img_size, CFG.img_size))
def img_preprocessor(fpath):
    img = read_image(fpath)
    img = resizer(img)
    return img

# create image feature vector
img_feature = np.stack([img_preprocessor(i) for i in df_full["img_path"]])
img_feature = img_feature.astype("float32")
img_feature.shape

(993, 3, 384, 384)

In [14]:
%%time

target_model_params = {target: [].copy() for target in label_mapper[CFG.img_category]}
target_model = {target: [].copy() for target in label_mapper[CFG.img_category]}
target_score = {target: [].copy() for target in label_mapper[CFG.img_category]}

for fold in range(CFG.n_folds):
    print(f"\n\n===== FOLD {fold} =====\n\n")

    seed_everything(fold)

    # training on each targets
    for target in tqdm(label_mapper[CFG.img_category]):

        # create feature
        train_x = img_feature[df_full[f"fold_{target}"] != fold].copy()
        valid_x = img_feature[df_full[f"fold_{target}"] == fold].copy()
        train_y = df_full.loc[df_full[f"fold_{target}"] != fold, target].values
        valid_y = df_full.loc[df_full[f"fold_{target}"] == fold, target].values

        # create model & training
        model_params = {
            "n_input": 1024,
            "hidden_layers": 256,
            "n_classes": len(label_mapper[CFG.img_category][target].keys()) if len(label_mapper[CFG.img_category][target].keys()) > 2 else 1,
        }

        model_threshold_params = {
            "fixed_params": {
                "criterion": "gini",
            },
            "dynamic_params": {
                "max_depth": [2, 4, 6],
                "min_samples_leaf": [1, 3, 5, 10, 20],
            },           
        }
        
        model = DNN_TreeClassifier(model_params, model_threshold_params, model_backbone, feature_extractor)
        model.fit(train_x, train_y, valid_x, valid_y)

        output = model.predict_proba(valid_x)
        
        score = {
            "logloss": skl_metrics.log_loss(valid_y, output["y_prob"]),
            "accuracy": skl_metrics.accuracy_score(valid_y,  output["y_prob_threshold_opt"].argmax(axis=1)),
            "f1": skl_metrics.f1_score(valid_y, output["y_prob_threshold_opt"].argmax(axis=1), average="macro"),
        }

        target_model_params[target].append(model_params)
        target_model[target].append(model)
        target_score[target].append(score)

    #     break

    # break

    if CFG.debug:
        break



===== FOLD 0 =====




  0%|          | 0/6 [00:00<?, ?it/s]

INFO: number of iteration : 198

Epoch[1/30]
 train loss : 0.18372
 train accuracy : 0
 train f1 : 0
 valid loss : 0.16579
 valid accuracy : 0
 valid f1 : 0
 eta : 0.00049
 Elapsed : 43.333

INFO: Found best weight


Epoch[2/30]
 train loss : 0.15098
 train accuracy : 0
 train f1 : 0
 valid loss : 0.1412
 valid accuracy : 0
 valid f1 : 0
 eta : 0.00048
 Elapsed : 42.458

INFO: Found best weight


Epoch[3/30]
 train loss : 0.13906
 train accuracy : 0
 train f1 : 0
 valid loss : 0.1366
 valid accuracy : 0
 valid f1 : 0
 eta : 0.00047
 Elapsed : 42.304

INFO: Found best weight


Epoch[4/30]
 train loss : 0.12865
 train accuracy : 0
 train f1 : 0
 valid loss : 0.1582
 valid accuracy : 0
 valid f1 : 0
 eta : 0.00047
 Elapsed : 42.498

Epoch[5/30]
 train loss : 0.11561
 train accuracy : 0
 train f1 : 0
 valid loss : 0.14639
 valid accuracy : 0
 valid f1 : 0
 eta : 0.00046
 Elapsed : 41.677

Epoch[6/30]
 train loss : 0.1184
 train accuracy : 0
 train f1 : 0
 valid loss : 0.14744
 valid accura

 17%|█▋        | 1/6 [10:25<52:06, 625.33s/it]

INFO: number of iteration : 198

Epoch[1/30]
 train loss : 0.23382
 train accuracy : 0
 train f1 : 0
 valid loss : 0.19006
 valid accuracy : 0
 valid f1 : 0
 eta : 0.00049
 Elapsed : 43.255

INFO: Found best weight


Epoch[2/30]
 train loss : 0.19323
 train accuracy : 0
 train f1 : 0
 valid loss : 0.1816
 valid accuracy : 0
 valid f1 : 0
 eta : 0.00048
 Elapsed : 42.09

INFO: Found best weight


Epoch[3/30]
 train loss : 0.17764
 train accuracy : 0
 train f1 : 0
 valid loss : 0.23558
 valid accuracy : 0
 valid f1 : 0
 eta : 0.00047
 Elapsed : 42.714

Epoch[4/30]
 train loss : 0.17162
 train accuracy : 0
 train f1 : 0
 valid loss : 0.17513
 valid accuracy : 0
 valid f1 : 0
 eta : 0.00047
 Elapsed : 42.818

INFO: Found best weight


Epoch[5/30]
 train loss : 0.18025
 train accuracy : 0
 train f1 : 0
 valid loss : 0.17479
 valid accuracy : 0
 valid f1 : 0
 eta : 0.00046
 Elapsed : 43.337

INFO: Found best weight


Epoch[6/30]
 train loss : 0.17912
 train accuracy : 0
 train f1 : 0
 valid 

...

INFO: number of iteration : 198

Epoch[1/30]
 train loss : 0.85431
 train accuracy : 0
 train f1 : 0
 valid loss : 0.7093
 valid accuracy : 0
 valid f1 : 0
 eta : 0.00049
 Elapsed : 43.546

INFO: Found best weight


Epoch[2/30]
 train loss : 0.63845
 train accuracy : 0
 train f1 : 0
 valid loss : 0.66637
 valid accuracy : 0
 valid f1 : 0
 eta : 0.00048
 Elapsed : 45.375

INFO: Found best weight


Epoch[3/30]
 train loss : 0.61622
 train accuracy : 0
 train f1 : 0
 valid loss : 0.6453
 valid accuracy : 0
 valid f1 : 0
 eta : 0.00047
 Elapsed : 44.98

INFO: Found best weight


Epoch[4/30]
 train loss : 0.55938
 train accuracy : 0
 train f1 : 0
 valid loss : 0.6619
 valid accuracy : 0
 valid f1 : 0
 eta : 0.00047
 Elapsed : 43.798

Epoch[5/30]
 train loss : 0.56895
 train accuracy : 0
 train f1 : 0
 valid loss : 0.63445
 valid accuracy : 0
 valid f1 : 0
 eta : 0.00046
 Elapsed : 44.335

INFO: Found best weight


Epoch[6/30]
 train loss : 0.53366
 train accuracy : 0
 train f1 : 0
 valid lo

 83%|████████▎ | 5/6 [1:20:29<15:40, 940.74s/it] 

INFO: number of iteration : 198

Epoch[1/30]
 train loss : 0.26676
 train accuracy : 0
 train f1 : 0
 valid loss : 0.22768
 valid accuracy : 0
 valid f1 : 0
 eta : 0.00049
 Elapsed : 44.374

INFO: Found best weight


Epoch[2/30]
 train loss : 0.23143
 train accuracy : 0
 train f1 : 0
 valid loss : 0.20894
 valid accuracy : 0
 valid f1 : 0
 eta : 0.00048
 Elapsed : 43.223

INFO: Found best weight


Epoch[3/30]
 train loss : 0.21678
 train accuracy : 0
 train f1 : 0
 valid loss : 0.20716
 valid accuracy : 0
 valid f1 : 0
 eta : 0.00047
 Elapsed : 43.552

INFO: Found best weight


Epoch[4/30]
 train loss : 0.20532
 train accuracy : 0
 train f1 : 0
 valid loss : 0.20426
 valid accuracy : 0
 valid f1 : 0
 eta : 0.00047
 Elapsed : 44.512

INFO: Found best weight


Epoch[5/30]
 train loss : 0.20301
 train accuracy : 0
 train f1 : 0
 valid loss : 0.1936
 valid accuracy : 0
 valid f1 : 0
 eta : 0.00046
 Elapsed : 43.497

INFO: Found best weight


Epoch[6/30]
 train loss : 0.18098
 train accurac

100%|██████████| 6/6 [1:40:13<00:00, 1002.19s/it]

Wall time: 7h 52min 5s


## Validation score summary

In [15]:
df_score = pd.DataFrame(columns=["logloss", "accuracy", "f1"])
for target in label_mapper[CFG.img_category]:
    tmp = pd.DataFrame(target_score[target])
    tmp.to_csv(architecture_root_path + "score_" + target + ".csv", index=False)
    df_score.loc[target] = tmp.mean(axis=0)
display(df_score)
df_score.to_csv(architecture_root_path + "score_total.csv", index=False)

,logloss,accuracy,f1
eye_yn,0.136238,0.950667,0.607128
leg_yn,0.165808,0.941587,0.608331
loc,0.406082,0.847967,0.483054
mouth_yn,0.259208,0.896284,0.600228
size,0.532576,0.777428,0.758320
arm_yn,0.201751,0.925481,0.592713


## Define inference function & testing

In [16]:
class TreeClassifier():
    def __init__(self, model_list):
        self.model_list = model_list
    def predict_proba(self, img_path):
        # create feature
        img = np.stack([img_preprocessor(i) for i in img_path])
        # training on each targets
        output = {target: [].copy() for target in label_mapper[CFG.img_category]}
        for fold in range(CFG.n_folds):
            for target in tqdm(label_mapper[CFG.img_category]):
                output[target].append(self.model_list[target][fold].predict_proba(img))
        return output

In [17]:
tree_classifier = TreeClassifier(target_model)

In [18]:
output = tree_classifier.predict_proba(df_full["img_path"].iloc[:10].values)

100%|██████████| 6/6 [00:02<00:00,  2.12it/s]


In [19]:
test_prob = {}
test_pred = {}
for target in label_mapper[CFG.img_category]:
    test_prob[target] = np.stack([i["y_prob_threshold_opt"] for i in output[target]], axis=0).mean(axis=0)
    test_pred[target] = test_prob[target].argmax(axis=1)

In [20]:
test_pred

{'eye_yn': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64),
 'leg_yn': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64),
 'loc': array([0, 1, 2, 1, 2, 1, 1, 2, 2, 1], dtype=int64),
 'mouth_yn': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64),
 'size': array([1, 1, 1, 1, 1, 1, 2, 1, 2, 1], dtype=int64),
 'arm_yn': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)}

## Save data

In [21]:
pickleIO(target_model_params, architecture_root_path + "target_model_params.pkl", "w")
pickleIO(target_model, architecture_root_path + "target_model.pkl", "w")